#Bibliotecas

In [ ]:
#Importando o tensorflow
import tensorflow as tf
#Importanto um leitor e prepocessador de imagens para lermos imagens de um diretório
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Importanto o matplotlib para plotar as imagens
import matplotlib.pyplot as plt
#Importanto o numpy para fazer algumas operações nos vetores das imagens
import numpy as np

#Instanciando um Gerador de Imagens

In [ ]:
diretorio_treinamento = '/content/drive/MyDrive/Datasets/Imagens/cats_and_dogs/train'
diretorio_teste = '/content/drive/MyDrive/Datasets/Imagens/cats_and_dogs/test'

In [ ]:
#Utilizando o data aumenta do ImageDataGenerator
data_train_generator = ImageDataGenerator(rescale=1/255.0, rotation_range=20, width_shift_range=0.1, 
                              height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, 
                              horizontal_flip=True, fill_mode="nearest")
#Criando um gerador de teste a parte para não criar um conjunto de teste diferente da solução anterior
data_test_generator = ImageDataGenerator(rescale=1/255.0)

In [ ]:
gerador_treino = data_train_generator.flow_from_directory(diretorio_treinamento, batch_size=32, target_size=(224,224), class_mode='sparse')

In [ ]:
gerador_teste = data_test_generator.flow_from_directory(diretorio_teste, batch_size=32, target_size=(224,224), class_mode='sparse')

#Utilizando um Modelo Pré-treinado

In [ ]:
#Include_top = False significa que não estamos importando a parte final da redes (camadas densas)
#Utilizando os parâmetros treinados para a base Imagenet (weights='imagenet')
model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')

In [ ]:
#Congelando a parte convolucional da rede já que não foi importada a camada densa
model.trainable = False

In [ ]:
#Criando camadas para a parte final da rede e linkando com a ultima camada do modelo base
average_layer = tf.keras.layers.GlobalAveragePooling2D()(model.output)
hidden_dense = tf.keras.layers.Dense(128,activation='relu')(average_layer)

In [ ]:
#Criando uma camada de Dropout entre a a primeira camada densa e a camada de saída
#Os neurônios terão 20% de probabilidade de serem "dropados" da iteração
dropout = tf.keras.layers.Dropout(0.2)(hidden_dense)

In [ ]:
#Criando a camada de predição e criando o modelo unindo as partes
preds = tf.keras.layers.Dense(1,activation='sigmoid')(dropout)
model_final = tf.keras.models.Model(inputs=model.input,outputs=preds)

#Sumário do Modelo

In [ ]:
model_final.summary()

# Compilando o Modelo

In [ ]:
model_final.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinando o Modelo

In [ ]:
model_final.fit(gerador_treino, epochs=10)

# Avaliando o Modelo

In [ ]:
loss, accuracy = model_final.evaluate(gerador_teste)

In [ ]:
loss

In [ ]:
accuracy

# Vendo o resultado na prática

In [ ]:
plt.imshow(gerador_teste[2][0][0])

In [ ]:
teste_imagem = gerador_teste[2][0][0].reshape(1,224,224,3)

In [ ]:
# A priori, como é uma classificação binária, podemos definir um limiar < 0.5 = gato e >= 0.5 cachoro
model_final.predict(teste_imagem)

In [ ]:
plt.imshow(gerador_teste[9][0][0])

In [ ]:
teste_imagem2 = gerador_teste[9][0][0].reshape(1,224,224,3)

In [ ]:
model_final.predict(teste_imagem2)